In [61]:
import requests
from bs4 import BeautifulSoup
import csv
import json

In [62]:
main_url = "https://myanimelist.net/topanime.php"

In [63]:
urls = []

Get all urls of anime

In [64]:
def get_urls(l):
    try:
        response = requests.get(l)
        soup = BeautifulSoup(response.content,"html.parser")
        links = soup.select(".anime_ranking_h3 .hoverinfo_trigger")

        for link in links:
            urls.append(link["href"])

        # next page
        url_next = main_url + soup.select_one(".pagination .next")["href"]
        # print(url_next)
        get_urls(url_next)
    except:
        print("Out of page")

Run get_urls() function

In [65]:
get_urls(main_url)

Out of page


Write urls anime to json file

In [66]:
# with open("animes_urls.json","w") as f:
#     json.dump(urls,f,indent=4)

Init animes array

In [75]:
animes = []

Get all anime detail from urls

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}


for id,url in enumerate(urls):
    try:
        response = requests.get(url, headers=headers, timeout=10) 
        response.raise_for_status()
        if response.status_code == 200:
            soup = BeautifulSoup(response.content,"html.parser")

            anime = {}
            anime["id"] = id + 1
            anime["url"] = url
            
            title_element = soup.select_one(".title-name")
            if title_element:
                anime["title"] = title_element.text
            else:
                anime["title"] = "N/A"  # or some default value

            image_element = soup.select_one(".lazyload")
            if image_element and "data-src" in image_element.attrs:
                anime["image"] = image_element["data-src"]
            else:
                anime["image"] = "N/A"  # or some default value

            for info in soup.select(".leftside .spaceit_pad"):
                key = value = ""
                if info.select_one(".dark_text"):
                    key = info.select_one(".dark_text").text.replace(":","")
                    value = info.text.replace(info.select_one(".dark_text").text,"").replace('"', '\\"').strip()
                else:
                    key = "None"
                    value = "None"
                
                anime[key] = value
            print(anime)
            animes.append(anime)
    except requests.exceptions.Timeout:
        print(f"Error at {url} detail: Request timed out")
    except requests.exceptions.RequestException as e:
        print(f"Error at {url} detail: {e}")
    except Exception as error:
        print(f"Error at {url} detail: {error}")

In [ ]:
# fields = [key for key,value in animes[0].items()]
# print(fields)
# # write csv
# with open("animes.csv", "w", encoding="utf-8", newline='') as f:
#     writer = csv.DictWriter(f, fieldnames=fields)
#     writer.writeheader()
#     writer.writerows(animes)

In [78]:
with open("animes.json","w") as f:
    json.dump(animes,f,indent=4)